In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import pickle 
import gc
import os
import time
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA 
import matplotlib.pyplot as plt

# import the LabelEncoder from sklearn
from sklearn.preprocessing import LabelEncoder

balance_data = False

In [2]:
# load data
train_data_df = pd.read_csv('data/train_cleaned.csv') # cleaned data gebruiken
test_data_df = pd.read_csv('data/test_cleaned.csv') # cleaned data gebruiken

# print missing values
print(train_data_df.isnull().sum())

c:\Users\agnes\anaconda3_nieuw\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


srch_id                        0
month                          0
site_id                        0
visitor_location_country_id    0
prop_country_id                0
prop_id                        0
prop_starrating                0
prop_review_score              0
prop_brand_bool                0
prop_log_historical_price      0
position                       0
price_usd                      0
promotion_flag                 0
srch_length_of_stay            0
srch_booking_window            0
srch_adults_count              0
srch_children_count            0
srch_room_count                0
srch_saturday_night_bool       0
orig_destination_distance      0
random_bool                    0
mean_position                  0
perc_clicked                   0
perc_booked                    0
target                         0
rating_relto_region            0
rating_relto_search            0
review_relto_search            0
review_relto_region            0
price_per_night                0
price_rel_

In [3]:
train_data = train_data_df 
test_data = test_data_df

In [4]:
# print columns of train data
print(train_data.columns)

Index(['srch_id', 'month', 'site_id', 'visitor_location_country_id',
       'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score',
       'prop_brand_bool', 'prop_log_historical_price', 'position', 'price_usd',
       'promotion_flag', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'random_bool',
       'mean_position', 'perc_clicked', 'perc_booked', 'target',
       'rating_relto_region', 'rating_relto_search', 'review_relto_search',
       'review_relto_region', 'price_per_night', 'price_rel_to_region',
       'price_rel_to_search', 'rel_distance', 'prop_location_score',
       'comb_rate', 'comb_inv'],
      dtype='object')


In [5]:
# print columns of test data
print(test_data.columns)

Index(['srch_id', 'month', 'site_id', 'visitor_location_country_id',
       'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score',
       'prop_brand_bool', 'prop_log_historical_price', 'price_usd',
       'promotion_flag', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'random_bool',
       'mean_position', 'perc_clicked', 'perc_booked', 'rating_relto_region',
       'rating_relto_search', 'review_relto_search', 'review_relto_region',
       'price_per_night', 'price_rel_to_region', 'price_rel_to_search',
       'rel_distance', 'prop_location_score', 'comb_rate', 'comb_inv'],
      dtype='object')


In [6]:
# check if test and train have same columns
print(set(train_data.columns) - set(test_data.columns))

# drop all columns that are not in test from train with exception of 'target'
drop_columns = set(train_data.columns) - set(test_data.columns) 
drop_columns.remove('target')
train_data = train_data.drop(drop_columns, axis=1)

# remove property id's from train and test (it is not a predictor, just an identifier)
train_data = train_data.drop(['prop_id'], axis=1)
test_prop = test_data['prop_id'] # save property id's for later use
test_data = test_data.drop(['prop_id'], axis=1)

{'position', 'target'}


In [7]:
# transform booleans and ordinal cat features to numeric ['prop_brand_bool', 'promotion_flag', 'srch_destination_id', 'srch_saturday_night_bool', 'random_bool']
train_data['prop_brand_bool'] = train_data['prop_brand_bool'].astype(int)
train_data['promotion_flag'] = train_data['promotion_flag'].astype(int)
train_data['srch_saturday_night_bool'] = train_data['srch_saturday_night_bool'].astype(int)
train_data['random_bool'] = train_data['random_bool'].astype(int)

In [8]:
test_data = test_data.drop('site_id', axis=1)
test_data = test_data.drop('visitor_location_country_id', axis=1)
test_data = test_data.drop('prop_country_id', axis=1)
train_data = train_data.drop('site_id', axis=1)
train_data = train_data.drop('visitor_location_country_id', axis=1)
train_data = train_data.drop('prop_country_id', axis=1)

In [9]:
# We cannot use one hot encoding here sadly, because we have too many unique values in the categorical features
categorical_features = ['month']        # TODO may become more if we include above deleted features

# first encode month to representimg integers (1-12) instead of names
months = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
train_data['month'] = train_data['month'].map(months)
test_data['month'] = test_data['month'].map(months)

# encode with cyclic encoding
train_data['month'] = train_data['month'].apply(lambda x: np.sin(x*(2.*np.pi/12)))
test_data['month'] = test_data['month'].apply(lambda x: np.sin(x*(2.*np.pi/12)))


In [10]:
# print what columns are missing in test data compared to train data
print('Columns missing in test data compared to train data: ', set(train_data.columns) - set(test_data.columns))

Columns missing in test data compared to train data:  {'target'}


In [11]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X_train = train_data.drop('target', axis=1)
y_train = train_data['target']

# balance data    
if balance_data:
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=42)

In [12]:
# create datasets for lightgbm
groups_train = X_train.groupby('srch_id').size().values
groups_valid = X_valid.groupby('srch_id').size().values
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features, group=groups_train, free_raw_data=False)
lgb_valid = lgb.Dataset(X_valid, y_valid, categorical_feature=categorical_features, group=groups_valid, free_raw_data=False)

In [13]:
import lightgbm as lgb

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [5],
    'force_col_wise': True,
    'is_unbalance': True,
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                verbose_eval=10, # GET NDCG@5 EVALUATION SCORE EVERY 10 ROUNDS 
                valid_sets=[lgb_train, lgb_valid],
                valid_names=['train', 'valid'])

Starting training...


c:\Users\agnes\anaconda3_nieuw\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\Users\agnes\anaconda3_nieuw\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 5161
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 30


c:\Users\agnes\anaconda3_nieuw\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\Users\agnes\anaconda3_nieuw\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[10]	train's ndcg@5: 0.688796	valid's ndcg@5: 0.946817
[20]	train's ndcg@5: 0.692278	valid's ndcg@5: 0.947423
[30]	train's ndcg@5: 0.695575	valid's ndcg@5: 0.948075
[40]	train's ndcg@5: 0.698772	valid's ndcg@5: 0.948706
[50]	train's ndcg@5: 0.701988	valid's ndcg@5: 0.949319
[60]	train's ndcg@5: 0.704438	valid's ndcg@5: 0.949776
[70]	train's ndcg@5: 0.706426	valid's ndcg@5: 0.950171
[80]	train's ndcg@5: 0.708078	valid's ndcg@5: 0.950442
[90]	train's ndcg@5: 0.709642	valid's ndcg@5: 0.950688
[100]	train's ndcg@5: 0.710996	valid's ndcg@5: 0.950893


In [18]:
# Get feature importance
feature_importance = gbm.feature_importance()
feature_names = gbm.feature_name()
print('Feature importances:', feature_importance)
print('Feature names:', feature_names)
# Sort feature importance in descending order
sorted_indices = np.argsort(feature_importance)[::-1]

# Select the top K features
K = 10  # Select top K features
selected_features = np.take(feature_names, sorted_indices[:K])

# Print the selected features
print("Selected features:")
print(selected_features)

# make sure srch id is still in the data
if 'srch_id' not in selected_features:
    selected_features = np.append(selected_features, 'srch_id')

lgb_train = lgb.Dataset(X_train[selected_features], y_train, group=groups_train, free_raw_data=False)
lgb_valid = lgb.Dataset(X_valid[selected_features], y_valid, group=groups_valid, free_raw_data=False)


Feature importances: [  1   0  63  37   7 282 116  28  48  54  39  22  31   8  58 136  83 275
 225 126 220 129  58  80  57 334  67 379  26  11]
Feature names: ['srch_id', 'month', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_log_historical_price', 'price_usd', 'promotion_flag', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'orig_destination_distance', 'random_bool', 'mean_position', 'perc_clicked', 'perc_booked', 'rating_relto_region', 'rating_relto_search', 'review_relto_search', 'review_relto_region', 'price_per_night', 'price_rel_to_region', 'price_rel_to_search', 'rel_distance', 'prop_location_score', 'comb_rate', 'comb_inv']
Selected features:
['prop_location_score' 'price_rel_to_search' 'prop_log_historical_price'
 'perc_clicked' 'perc_booked' 'rating_relto_search' 'random_bool'
 'review_relto_search' 'rating_relto_region' 'price_usd']


In [19]:
# perform lightgbm training again with selected features
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                verbose_eval=10, # GET NDCG@5 EVALUATION SCORE EVERY 10 ROUNDS
                valid_sets=[lgb_train, lgb_valid],
                valid_names=['train', 'valid'])

c:\Users\agnes\anaconda3_nieuw\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\Users\agnes\anaconda3_nieuw\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


TypeError: Wrong type(str) or unknown name(month) in categorical_feature

In [ ]:
# make predictions
y_pred = gbm.predict(test_data, num_iteration=gbm.best_iteration)

In [ ]:
# rank predictions per search id
test_data['pred'] = y_pred

# re-add prop id
test_data['prop_id'] = test_prop

# sort predictions by search id and prediction
test_data = test_data.sort_values(['srch_id', 'pred'], ascending = [True, False])

In [ ]:
df = test_data[['srch_id', 'prop_id']]
df['srch_id'] = df['srch_id'].astype('int')
df['prop_id'] = df['prop_id'].astype('int')

# drop index
df = df.reset_index(drop=True)
df.head()

In [ ]:
# save predictions to csv
if balance_data:
    df.to_csv('submission/submission_gbm_FF_balanced.csv', index=False)
    
else:
    df.to_csv('submission/submission_gbm_FF.csv', index=False)